In [1]:
from datetime import datetime
import pymc3 as pm
import pandas as pd
import numpy as np
import warnings
import theano.tensor as tt
from src.get_data import Company

from bokeh.plotting import figure, show
from bokeh.models import BoxAnnotation, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.palettes import brewer
output_notebook()

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Loading BokehJS ...

# Data Manipulation Time

In [2]:
class CashMoneySwag():
    def __init__(self, ticker, start_date=datetime(2000,1,1), end_date=datetime.today()):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date

        self.comp = Company(ticker, self.start_date, self.end_date)
        self.data = self._prep_data()
        self.split_date = self.data_early = self.data_later = None

    def _prep_data(self):
        self.comp.data_frame.index = pd.to_datetime(self.comp.data_frame.Date)
        raw_y = self.comp.return_numpy_array_of_company_daily_stock_close()
        norm_y = (raw_y-raw_y[0])/np.std(raw_y)
        t = self._dates_to_idx(self.comp.data_frame.index)
        return pd.DataFrame(data={'t':t, 'norm_y':norm_y, 'raw_y': raw_y},index=self.comp.data_frame.index)

    def go(self, start_date=None,split_date=pd.to_datetime("2019-09-30"),end_date=None):
        self._gen_test_train_data(start_date, split_date, end_date)
        self._train_gp()
        self._predict_gp()
        return self._get_plot_vals()

    def _gen_test_train_data(self, start_date, split_date, end_date):
        if start_date==None:
            start_date=self.start_date
        if end_date==None:
            end_date=self.end_date
        self.train_start = start_date
        self.test_end = end_date
        self.split_date=split_date
        start_idx = self.data.index.searchsorted(start_date)
        sep_idx = self.data.index.searchsorted(split_date) # Y-M-D
        end_idx = self.data.index.searchsorted(end_date)
        self.data_early = self.data.iloc[start_idx:sep_idx, :]
        self.data_later = self.data.iloc[sep_idx:end_idx+1, :]

    def _train_gp(self):
        with pm.Model() as model:
            # yearly periodic component x long term trend
            η_per = pm.HalfCauchy("η_per", beta=0.75, testval=1.0)
            period  = pm.Normal("period", mu=1, sigma=0.05)
            ℓ_psmooth = pm.Gamma("ℓ_psmooth ", alpha=4, beta=3)
            cov_seasonal = η_per**2 * pm.gp.cov.Periodic(1, period, ℓ_psmooth)
            gp_seasonal = pm.gp.Marginal(cov_func=cov_seasonal)

            # small/medium term irregularities
            η_med = pm.HalfCauchy("η_med", beta=0.5, testval=0.1)
            ℓ_med = pm.Gamma("ℓ_med", alpha=4, beta=7)
            α = pm.Gamma("α", alpha=5, beta=2)
            cov_medium = η_med**2 * pm.gp.cov.RatQuad(1, ℓ_med, α)
            gp_medium = pm.gp.Marginal(cov_func=cov_medium)

            # long term trend
            η_trend = pm.HalfCauchy("η_trend", beta=1, testval=2.0)
            ℓ_trend = pm.Gamma("ℓ_trend", alpha=7, beta=3)
            cov_trend = η_trend**2 * pm.gp.cov.ExpQuad(1, ℓ_trend)
            gp_trend = pm.gp.Marginal(cov_func=cov_trend)

            # noise model
            η_noise = pm.HalfNormal("η_noise", sigma=0.5, testval=0.05)
            ℓ_noise = pm.Gamma("ℓ_noise", alpha=2, beta=4)
            σ  = pm.HalfNormal("σ",  sigma=0.25, testval=0.05)
            cov_noise = η_noise**2 * pm.gp.cov.Matern32(1, ℓ_noise) +\
                        pm.gp.cov.WhiteNoise(σ)

            # The Gaussian process is a sum of these three components
            self.gp = gp_seasonal + gp_medium + gp_trend

            # Since the normal noise model and the GP are conjugates
            # We use `Marginal` with the `.marginal_likelihood` method
            t=self.data_early['t'].values[:,None]
            y=self.data_early['norm_y'].values
            y_ = self.gp.marginal_likelihood("y", X=t, y=y, noise=cov_noise)

            # this line calls an optimizer to find the MAP
            self.mp = pm.find_MAP(include_transformed=True)

    def _predict_gp(self):
        # predict at a 15 day granularity
        dates = pd.date_range(start=self.train_start, end=self.test_end, freq="5D")
        tnew = self._dates_to_idx(dates)[:,None]
        std_y=np.std(self.data['raw_y'].values)
        first_y=self.data['raw_y'][0]

        print("Predicting with gp ...")
        mu, var = self.gp.predict(tnew, point=self.mp, diag=True)
        mean_pred = mu*std_y + first_y
        var_pred  = var*std_y**2

        # make dataframe to store fit results
        self.fit = pd.DataFrame({"t": tnew.flatten(),
                            "mu_total": mean_pred,
                            "sd_total": np.sqrt(var_pred)},
                           index=dates)

    def _get_plot_vals(self):
        fit = self.fit
        upper = fit.mu_total + 2*fit.sd_total
        lower = fit.mu_total - 2*fit.sd_total
        band_x = np.append(fit.index.values, fit.index.values[::-1])
        band_y = np.append(lower, upper[::-1])
        xy_pred = [fit.index, fit.mu_total, fit.sd_total]
        std_bounds = [band_x, band_y]
        train_data = [self.data_early.index, self.data_early['raw_y']]
        test_data = [self.data_later.index, self.data_later['raw_y']]
        return xy_pred, std_bounds, train_data, test_data

    def _plot_stock_data(self,show_split=False,raw_data=True,rg=None):
        # If no range specified, assume whole thing
        if rg==None:
            rg=[self.start_date,self.end_date]
        # Prep data to plot
        s_idx=self.data.index.searchsorted(rg[0])
        e_idx=self.data.index.searchsorted(rg[1])
        sel_dat=self.data.iloc[s_idx:e_idx,:]
        t=sel_dat.index
        if raw_data:
            y=sel_dat['raw_y']
        else:
            y=sel_dat['norm_y']
        # Plot data
        p = figure(x_axis_type='datetime', title='Prices of '+self.ticker+' stock over time',
                   x_range=rg, plot_width=900, plot_height=573)
        p.yaxis.axis_label = 'Stock price'
        p.xaxis.axis_label = 'Date'
        p.line(t,y,line_width=2, line_color="black", alpha=0.5)
        # Add box region to show test/train split
        if show_split and self.split_date==None:
            warnings.warn('Test/train split on data has not yet been performed.'+\
                          ' Ignoring request for test region highlighting.')
        elif show_split and self.split_date!=None:
            predict_region = BoxAnnotation(left=self.split_date,
                               fill_alpha=0.1, fill_color="yellow")
            p.add_layout(predict_region)
        show(p)

    def _dates_to_idx(self, tlist):
        reference_time = self.start_date
        t = (tlist - reference_time) / pd.Timedelta(30, "D")
        return np.asarray(t)

In [3]:
cms = CashMoneySwag('AAPL')
rg=[datetime(2019,7,31),datetime(2019,9,30),datetime(2019,10,31)]
xy_pred, std_bounds, train_data, test_data = cms.go(start_date=rg[0],split_date=rg[1],end_date=rg[2])

Loaded data for AAPL from 2000-01-01 00:00:00 to 2019-12-07 20:48:30.542601 from internet


/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the f

Predicting with gp ...


/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/home/gen/.virtualenvs/pybar/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the f

In [6]:
rg=[datetime(2019,5,31),datetime(2019,7,31),datetime(2019,8,31)]
xy_pred, std_bounds, train_data, test_data = cms.go(start_date=rg[0],split_date=rg[1],end_date=rg[2])

logp = 72.869, ||grad|| = 0.0029925: 100%|██████████| 74/74 [00:00<00:00, 142.96it/s]  


Predicting with gp ...


In [7]:
# Set up plot
p = figure(plot_height=572, plot_width=900,
              title="my sine wave",x_axis_type='datetime',
              tools="crosshair,pan,reset,save,wheel_zoom")
p.yaxis.axis_label = 'money'
p.xaxis.axis_label = 'time'

# Set up source data
sfit = ColumnDataSource(data=dict(x=xy_pred[0], y=xy_pred[1]))
bfit = ColumnDataSource(data=dict(x=std_bounds[0], y=std_bounds[1]))
defit = ColumnDataSource(data=dict(x=train_data[0], y=train_data[1]))
dlfit = ColumnDataSource(data=dict(x=test_data[0], y=test_data[1]))

# Plot total fit
p.line('x','y',source=sfit,
       line_width=1, line_color="firebrick", legend="Total fit")
p.patch('x', 'y',source=bfit,
        color="firebrick", alpha=0.6, line_color="white")

# Plot true value
p.circle('x','y',source=defit,
         color="black", legend="Training data")
p.circle('x','y',source=dlfit,
         color="yellow", legend="Test data")
p.legend.location = "top_left"

show(p)

In [50]:
cms.plot_stock_data(show_split=True,raw_data=True,rg=[rg[0], rg[2]])

 * Rising trend (ExpQuad)
 * Periodic jumps, shorter term than trend (Periodic)
 * Irregularities (short term, Rational Quadratic)
 * Noise kernel (Exp + White Noise)

# Cowabunga

In [51]:
## plot the components
p = figure(title="Predicate",
           x_axis_type='datetime', plot_width=900, plot_height=572)
p.yaxis.axis_label = 'money'
p.xaxis.axis_label = 'time'

In [52]:
# plot mean and 2σ region of total prediction
xy_pred, std_bounds, train_data, test_data = cms.get_plot_vals()

sfit = ColumnDataSource(data=dict(x=xy_pred[0], y=xy_pred[1]))
bfit = ColumnDataSource(data=dict(x=std_bounds[0], y=std_bounds[1]))
defit = ColumnDataSource(data=dict(x=train_data[0], y=train_data[1]))
dlfit = ColumnDataSource(data=dict(x=test_data[0], y=test_data[1]))
# total fit
p.line('x','y',source=sfit,
       line_width=1, line_color="firebrick", legend="Total fit")
p.patch('x', 'y',source=bfit,
        color="firebrick", alpha=0.6, line_color="white")

# true value
p.circle('x','y',source=defit,
         color="black", legend="Training data")
p.circle('x','y',source=dlfit,
         color="yellow", legend="Test data")
p.legend.location = "top_left"
show(p)

# Prior exploration

In [5]:
x = np.linspace(0, 150, 5000)
priors = [
    ("ℓ_psmooth", pm.Gamma.dist(alpha=4,  beta=3)),
    ("period",    pm.Normal.dist(mu=1.0,  sigma=0.05)),
    ("ℓ_med",     pm.Gamma.dist(alpha=4,  beta=7)),
    ("α",         pm.Gamma.dist(alpha=5,  beta=2)),
    ("ℓ_trend",   pm.Gamma.dist(alpha=7,  beta=3)),
    ("ℓ_noise",   pm.Gamma.dist(alpha=2,  beta=4))]

colors = brewer['Paired'][6]

p = figure(title='Lengthscale and period priors',
           plot_width=550, plot_height=350,  x_range=(-1, 8), y_range=(0, 2))
p.yaxis.axis_label = 'Probability'
p.xaxis.axis_label = 'Months'

for i, prior in enumerate(priors):
    p.line(x, np.exp(prior[1].logp(x).eval()), legend_label=prior[0],
           line_width=3, line_color=colors[i])
show(p)

In [6]:
x = np.linspace(0, 4, 5000)
priors = [
    ("η_per",   pm.HalfCauchy.dist(beta=0.75)),
    ("η_med",   pm.HalfCauchy.dist(beta=0.5)),
    ("η_trend", pm.HalfCauchy.dist(beta=1)), # will use beta=2, but 2.2 is visible on plot
    ("σ",       pm.HalfNormal.dist(sigma=0.25)),
    ("η_noise", pm.HalfNormal.dist(sigma=0.5))]

colors = brewer['Paired'][5]

p = figure(title='Scale priors',
           plot_width=550, plot_height=350)
p.yaxis.axis_label = 'Probability'
p.xaxis.axis_label = 'Years'

for i, prior in enumerate(priors):
    p.line(x, np.exp(prior[1].logp(x).eval()), legend_label=prior[0],
           line_width=3, line_color=colors[i])
show(p)

In [7]:
t=cms.data_early['t'].values[:,None]
y=cms.data_early['norm_y'].values

In [8]:
cms.data_early

,t,norm_y,raw_y
Date,,,
2019-07-31,234.945276,3.362077,213.039993
2019-08-01,234.978131,3.287934,208.429993
2019-08-02,235.010986,3.217007,204.020004
2019-08-05,235.109551,3.045237,193.339996
2019-08-06,235.142405,3.104102,197.000000
2019-08-07,235.175260,3.136912,199.039993
2019-08-08,235.208115,3.207517,203.429993
2019-08-09,235.240970,3.168274,200.990005
2019-08-12,235.339535,3.160072,200.479996
